In [ ]:
# imports

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
import seaborn as sns
from scipy import stats
import networkx as nx
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV

In [ ]:
# reading and looking at the dataset

df = pd.read_csv('Fortune_500_2017.csv')
df.info()
df.describe()

In [ ]:
# changing profit change from string to float

df['Prftchange'] = df['Prftchange'].apply(lambda x: float(x.replace(',','')))

In [ ]:
# chacking to see how many ceos only appeared once
df['Ceo'].nunique()

# seeing who appeared more than once
a = df[df['Ceo'].duplicated(keep=False)]
a[['Rank','Ceo', 'Title']]

In [ ]:
# chacking to see if there are any null values

df.isnull().any().any()

In [ ]:
# graph of revenue by rank

plt.figure(figsize=(14,7))
ax1 = plt.axes() # standard axes
ax2 = plt.axes([0.35, 0.35, 0.45, 0.45])
ax1.set_xlabel("Rank")
ax1.set_ylabel("Revenue")
ax2.set_xlabel("Top 10%")
ax1.plot(df['Revenues'], '--o')
ax2.plot(df['Revenues'].iloc[0:51], '--bo', label='Revenues')
mark_inset(ax1, ax2, loc1=1, loc2=4, fc="none", ec="0.5")
mark_inset(ax1, ax2, loc1=2, loc2=3, fc="none", ec="0.5")
ax2.legend()
plt.savefig('Revenue.png',bbox_inches='tight')

In [ ]:
# feature creation:

# Growth direction: if profit has increased or decreased

def growth_direction (row):
   num = row['Prftchange']
   if num > 0:
      return 'Positive'
   elif num == 0:
      return 'Same'  
   else:
       return 'Negative'

# Ranking group: ranking ranges based on distribution of revenue
def ranking_group (row):
   rank = int(row['Rank'])
   if rank <= 15:
      return '0-3'
   elif rank <= 30:
      return '3-6'  
   elif rank <= 50:
      return '6-10'
   elif rank <= 100:
      return '10-20'
   elif rank <= 200:
      return '20-40'
   elif rank <= 300:
      return '40-60'
   elif rank <= 400:
      return '60-80'
   else:
      return '80-100'

# Revenue per employee
df['Revenue_per_employee'] = df.apply(lambda row: row.Revenues / row.Employees, axis=1)

df['Ranking_group'] = df.apply(lambda row: ranking_group(row), axis=1)
df['Growth_direction'] = df.apply(lambda row: growth_direction(row), axis=1)

In [ ]:
# show dataset

df.head()

In [ ]:
# graph of distribution of assets

df['Assets'].plot(kind="box")

In [ ]:
# creating normalised dataset for graphs

normal_df = df.copy()
x, y = df.Revenues.mean(), df.Revenues.std()
normal_df['Revenues'] = (df.Revenues - x) / y
x, y = df.Assets.mean(), df.Assets.std()
normal_df['Assets'] = (df.Assets - x) / y
x, y = df.Totshequity.mean(), df.Totshequity.std()
normal_df['Totshequity'] = (df.Totshequity - x) / y
x, y = df.Profits.mean(), df.Profits.std()
normal_df['Profits'] = (df.Profits - x) / y
x, y = df.Prftchange.mean(), df.Prftchange.std()
normal_df['Prftchange'] = (df.Prftchange - x) / y
x, y = df.Revchange.mean(), df.Revchange.std()
normal_df['Revchange'] = (df.Revchange - x) / y
x, y = df.Revenue_per_employee.mean(), df.Revenue_per_employee.std()
normal_df['Revenue_per_employee'] = (df.Revenue_per_employee - x) / y
x, y = df.Employees.mean(), df.Employees.std()
normal_df['Employees'] = (df.Employees - x) / y
normal_df.head()

In [ ]:
# how many people are in each sector

ind = df[['Sector','Rank']].groupby('Sector').count()
ind['Rank'].sort_values().head(10)

In [ ]:
# how many ceos have each title 

ind = df[['Ceo-title','Rank']].groupby('Ceo-title').count()
ind['Rank'].sort_values().head(20)

In [ ]:
# graph of distribution by hq state

state = df['Hqstate'].value_counts(ascending=False)
plt.style.use('ggplot')
plt.figure(figsize=(15,5))
plt.ylabel("Count")
plt.xlabel("Hqstate")
state.plot(kind='bar')
plt.savefig('state.png',bbox_inches='tight')


In [ ]:
# graph of distribution by industry

prob = df['Industry'].value_counts(ascending=False)
plt.style.use('ggplot')
plt.figure(figsize=(15,5))
plt.ylabel("Count")
prob.plot(kind='bar')
plt.savefig('foo.png',bbox_inches='tight')

In [ ]:
# graph of profit (not revenue, to take into account expenses) for each industry

prof = df.groupby('Industry')['Profits'].mean()
plt.style.use('ggplot')
plt.figure(figsize=(20,7))
plt.ylabel("Profits")

prof.plot(kind='bar', color="darkgreen")
plt.savefig('Profits_by_Industry.png',bbox_inches='tight')

In [ ]:
# graph of distribution of revenue per employee based on rank

plt.figure(figsize=(25,12))
ax1 = plt.axes() 
ax1.set_xlabel("Rank")
ax1.set_ylabel("Revenue per Employee")
ax1.plot(df['Revenue_per_employee'], color="darkblue")
plt.savefig('Revenue_per_Employee.png',bbox_inches='tight')

In [ ]:
# graph of revenue by industry

rev = df.groupby('Industry')['Revenue_per_employee'].mean()
plt.style.use('ggplot')
plt.figure(figsize=(15,5))
plt.ylabel("Revenue")
rev.plot(kind='bar', color="violet")
plt.savefig('Revenue_per_Employee_for_Industry.png',bbox_inches='tight')

In [ ]:
# graph of revenue by industry without outlier

df['Industry'].loc[df['Industry'] == 'Miscellaneous']
df.iloc[394]
a = df.copy()
a = a.drop(394)
rev = a.groupby('Industry')['Revenue_per_employee'].mean()
plt.style.use('ggplot')
plt.figure(figsize=(15,5))
plt.ylabel("Revenue")
rev.plot(kind='bar', color="purple")
plt.savefig('Revenue_per_Employee_per_industry_without_misc.png',bbox_inches='tight')

In [ ]:
# graph of revenue by hq state

rev = df.groupby('Hqstate')['Revenues'].mean()
plt.style.use('ggplot')
plt.figure(figsize=(15,5))
plt.ylabel("Revenue")
rev.plot(kind='bar', color="orange")
plt.savefig('rev_per_state.png',bbox_inches='tight')

In [ ]:
# graph of revenue by hq state without outliers

b = df.iloc[5:]
rev = b.groupby('Hqstate')['Revenues'].mean()
plt.style.use('ggplot')
plt.figure(figsize=(15,5))
plt.ylabel("Revenue")
rev.plot(kind='bar', color="darkorange")
plt.savefig('rev_per_state_without_top_one_precent.png',bbox_inches='tight')

In [ ]:
# distribution graph of numerical data without rank and hqzip

a = df.drop(['Rank','Hqzip'], axis =1)
sns.pairplot(a)
plt.savefig('pairplot.png',bbox_inches='tight')

In [ ]:
# distribution graph of numerical data without rank, hqzip and profit change
# and without ouliers

a = df.drop(['Rank','Hqzip','Prftchange',], axis =1)
a = a.drop(a['Revenues'].idxmax())
a = a.drop(a['Revenue_per_employee'].idxmax())
sns.pairplot(a)
plt.savefig('pairplot2.png',bbox_inches='tight')

In [ ]:
# heatmap of correlations 
plt.figure(figsize = (10, 10))
sns.heatmap(df.drop(['Hqzip','Revenues','Rank'], axis =1).corr(), annot = True)
plt.savefig('correlation.png',bbox_inches='tight')

In [ ]:
# graph of distribution of growth direction by sector

direct = df.groupby('Sector')['Growth_direction'].value_counts(ascending=False).unstack(fill_value=0).stack()
direct = direct.reset_index()
direct = direct.rename(columns= {0: 'Count'})
direct.index.name = 'Index'
direct2 = direct.copy()
direct2['Growth_direction'] = direct2[['Sector', 'Growth_direction']].agg('-'.join, axis=1)
x, y = direct2.Count.min(), direct2.Count.max()
direct2['Count'] = (direct2.Count - x) / y - x
direct2['Precentage'] = 0

i = 0
while (i < len(direct2)):
    sum = direct2['Count'].iloc[i] + direct2['Count'].iloc[i+1] + direct2['Count'].iloc[i+2]  
    direct2['Precentage'].iloc[i] = (direct2['Count'].iloc[i] / sum) * 100
    direct2['Precentage'].iloc[i+1] = (direct2['Count'].iloc[i+1] / sum) * 100
    direct2['Precentage'].iloc[i+2] = (direct2['Count'].iloc[i+2] / sum) * 100
    i=i+3

direct2['Precentage'] = round(direct2['Precentage'],1)

colors_list = ['darkblue','lightblue', 'white']
plt.style.use('ggplot')
plt.figure(figsize=(30,8))
ax = direct2.plot(x="Growth_direction", y="Count", kind="bar", figsize=(30, 8), color = colors_list, legend=False)
ax.set_xlabel("Growth Direction by Sector")
ax.set_ylabel("Count") 
i=0
for bar in ax.patches:
    plt.text(bar.get_x() - 0.2 + bar.get_width() / 2, 
             bar.get_height()+0.01, direct2['Precentage'].iloc[i],rotation=90)
    i=i+1

plt.savefig('Growth_direction_by_Sector.png',bbox_inches='tight')

In [ ]:
# graph of distribution of growth direction by hq state

direct = df.groupby('Hqstate')['Growth_direction'].value_counts(ascending=False).unstack(fill_value=0).stack()
direct = direct.reset_index()
direct = direct.rename(columns= {0: 'Count'})
direct.index.name = 'Index'
direct2 = direct.copy()
direct2['Growth_direction'] = direct2[['Hqstate', 'Growth_direction']].agg('-'.join, axis=1)
x, y = direct2.Count.min(), direct2.Count.max()
direct2['Count'] = (direct2.Count - x) / y - x
direct2['Precentage'] = 0
i = 0
while (i < len(direct2)):
    sum = direct2['Count'].iloc[i] + direct2['Count'].iloc[i+1] + direct2['Count'].iloc[i+2]  
    direct2['Precentage'].iloc[i] = (direct2['Count'].iloc[i] / sum) * 100
    direct2['Precentage'].iloc[i+1] = (direct2['Count'].iloc[i+1] / sum) * 100
    direct2['Precentage'].iloc[i+2] = (direct2['Count'].iloc[i+2] / sum) * 100
    i=i+3

direct2['Precentage'] = round(direct2['Precentage'],1)

colors_list = ['darkblue','lightblue', 'white']
plt.style.use('ggplot')
plt.figure(figsize=(30,8))
ax = direct2.plot(x="Growth_direction", y="Count", kind="bar", figsize=(30, 8), color = colors_list, legend=False)
ax.set_xlabel("Growth Direction by Hqstate")
ax.set_ylabel("Count") 

i=0
for bar in ax.patches:
    plt.text(bar.get_x() - 0.2 + bar.get_width() / 2, 
             bar.get_height()+0.01, direct2['Precentage'].iloc[i],rotation=90)
    i=i+1

plt.savefig('Growth_direction_by_Hqstate.png',bbox_inches='tight')

In [ ]:
#  preparing data for clustering

X = normal_df.copy()
X = X[['Employees', 'Prftchange', 'Revchange']]

# outlier removal 
X=X.drop(X['Employees'].idxmax())
X=X.drop(X['Revchange'].idxmax())
X=X.drop(X['Prftchange'].idxmax())

In [ ]:
# OPTICS clustering (3 features)

clustering = OPTICS(min_samples=5).fit(X)

# plot of principle components 
plt.style.use('ggplot')
pca = PCA(n_components=2)
pca.fit(X)
X['PC1'] = pca.fit_transform(X)[:,0]
X['PC2'] = pca.fit_transform(X)[:,1]
X['OPTICS'] = clustering.labels_
print(clustering.labels_)
sns.scatterplot(data=X,x="PC1",y="PC2",hue=X['OPTICS'])
plt.savefig('components_optics2.png',bbox_inches='tight')

In [ ]:
# optics 3d plot

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Employees')
ax.set_ylabel('Prftchange')
ax.set_zlabel('Revchange')

for s in X.OPTICS.unique():
    ax.scatter(X.Employees[X.OPTICS==s], X.Prftchange[X.OPTICS==s], 
               X.Revchange[X.OPTICS==s],label=s)
   
ax.legend(loc='upper left')
print(X.OPTICS.unique())
plt.savefig('optics2.png',bbox_inches='tight')

In [ ]:
# DBSCAN clustering with 3d plot

clusters = DBSCAN(eps=0.5, min_samples=5).fit(X[['Employees', 'Prftchange', 'Revchange']])
X['DBSCAN'] = clusters.labels_

sns.set(style="whitegrid")
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Employees')
ax.set_ylabel('Prftchange')
ax.set_zlabel('Revchange')

for s in X.DBSCAN.unique():
    ax.scatter(X.Employees[X.DBSCAN==s], X.Prftchange[X.DBSCAN==s], 
               X.Revchange[X.DBSCAN==s],label=s)
ax.legend(loc='upper left')
print(X.DBSCAN.unique())
plt.savefig('dbscan2.png',bbox_inches='tight')

In [ ]:
# preparing data for clustering

X = normal_df.copy()
X = X[['Prftchange', 'Revchange']]

# outlier removal
X=X.drop(X['Revchange'].idxmax())
X=X.drop(X['Prftchange'].idxmax())
X=X.drop(X['Prftchange'].idxmax())

In [ ]:
# OPTICS clustering (2 features) with plot
clustering = OPTICS(min_samples=5).fit(X)
X['OPTICS'] = clustering.labels_

plt.figure(figsize=(14,7))
sns.scatterplot(data=X,x="Prftchange",y="Revchange",hue=X['OPTICS'])
plt.savefig('optics_revchange_profitchange.png',bbox_inches='tight')

In [ ]:
# DBSCAN clustering (2 features) with plot

clusters = DBSCAN(eps=0.5, min_samples=3).fit(X[['Prftchange', 'Revchange']])
X['DBSCAN'] = clusters.labels_

plt.figure(figsize=(14,7))
sns.scatterplot(data=X,x="Prftchange",y="Revchange",hue=X['DBSCAN'])
plt.savefig('dbscan_revchange_profitchange.png',bbox_inches='tight')

In [ ]:
# chi square test

contigency= pd.crosstab(df['Growth_direction'], df['Sector']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Growth_direction'], df['Industry']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Growth_direction'], df['Hqstate']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Growth_direction'], df['Ceo']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Growth_direction'], df['Ranking_group']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Hqstate'], df['Ranking_group']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Ceo'], df['Ranking_group']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Industry'], df['Ranking_group']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Sector'], df['Ranking_group']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))
contigency= pd.crosstab(df['Sector'], df['Industry']) 
c, p, dof, expected = stats.chi2_contingency(contigency) 
print(round(p,3))

In [ ]:
# prepare data for regression

data = df.drop(['Title', 'Website','Hqlocation', 'Hqaddr', 'Hqcity',
               'Ceo', 'Ceo-title','Address', 'Ticker', 'Fullname'], axis=1)

# drop outliers
data = data.drop(394)
data = data.drop(data['Revenues'].idxmax())
data = data.drop(data['Revenue_per_employee'].idxmax())

# changing qualitative data to quantitative
dummies = pd.get_dummies(data[['Sector', 'Industry','Hqstate']])

# preparing numerical columns
X_numerical = data.drop(['Sector', 'Industry', 'Hqstate', 'Hqstate', 'Hqzip',
                      'Hqtel', 'Revenues','Ranking_group', 'Growth_direction',
                       'Assets', 'Rank', 'Profits'], 
                        axis=1).astype('float64')
list_numerical = X_numerical.columns

# putting it all together
X = pd.concat([X_numerical, dummies], axis=1)

# what will be predicted
y = data['Revenues']

# splitting train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# normalizing data
scaler = StandardScaler().fit(X_train[list_numerical]) 
X_train[list_numerical] = scaler.transform(X_train[list_numerical])
X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [ ]:
# lasso regression with alpha = 1

reg = Lasso(alpha=1)
reg.fit(X_train, y_train)

print('R squared training set', round(reg.score(X_train, y_train)*100, 2))
print('R squared test set', round(reg.score(X_test, y_test)*100, 2))

# Training
pred_train = reg.predict(X_train)
mse_train = mean_squared_error(y_train, pred_train)
print('MSE training set', round(mse_train, 2))

# Test
pred = reg.predict(X_test)
mse_test =mean_squared_error(y_test, pred)
print('MSE test set', round(mse_test, 2))

In [ ]:
# plot of lasso with different alphas 

alphas = np.linspace(0.01,500,100)
lasso = Lasso(max_iter=10000)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train)
    coefs.append(lasso.coef_)

ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('Standardized Coefficients')
plt.savefig('lasso_coeff_alpha.png',bbox_inches='tight')

In [ ]:
# lasso with best alpha

# Lasso with 5 fold cross-validation
model = LassoCV(cv=5, random_state=0, max_iter=10000)

# Fit model
model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=10000, random_state=0)

lasso_best = Lasso(alpha=model.alpha_)
lasso_best.fit(X_train, y_train)

print('R squared training set', round(lasso_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_best.score(X_test, y_test)*100, 2))
mean_squared_error(y_test, lasso_best.predict(X_test))

In [ ]:
# print coefficients 

ser = pd.Series(lasso_best.coef_, index = X_train.columns)
print(ser[ser > 0])
print(ser[ser < 0])

In [ ]:
# ridge regression with alpha = 1
rig = Ridge(alpha=1)
rig.fit(X_train, y_train)
print('R squared training set', round(rig.score(X_train, y_train)*100, 2))
print('R squared test set', round(rig.score(X_test, y_test)*100, 2))

# Training 
pred_train_r = rig.predict(X_train)
mse_train_r = mean_squared_error(y_train, pred_train_r)
print('MSE training set', round(mse_train_r, 2))

# Test 
pred_r = rig.predict(X_test)
mse_test_r =mean_squared_error(y_test, pred_r)
print('MSE test set', round(mse_test_r, 2))


In [ ]:
# plot of lasso with different alphas 

alphas = np.linspace(0.01,500,100)
ridge = Ridge(max_iter=10000)
coefs = []


for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('Standardized Coefficients')
plt.savefig('ridge_coeff_alpha.png',bbox_inches='tight')


In [ ]:
# ridge with best alpha

# ridge with 5 fold cross-validation
model = RidgeCV(cv=5)

# Fit model
model.fit(X_train, y_train)

RidgeCV(cv=5)
print(model.alpha_)

ridge_best = Ridge(alpha=model.alpha_)
ridge_best.fit(X_train, y_train)

print('R squared training set', round(ridge_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(ridge_best.score(X_test, y_test)*100, 2))

mean_squared_error(y_test, ridge_best.predict(X_test))

In [ ]:
# ridge and lasso comparison graph

features = X.columns
target = 'Revenues'
print(features)
print(target)

plt.figure(figsize = (25, 10))
plt.plot(features,ridge_best.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,
         color='red',label=r'Ridge; $\alpha = 10$',zorder=7)
plt.plot(lasso_best.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,
         color='blue',label=r'lasso; $\alpha = 2019.8$')

plt.xticks(rotation = 90)
plt.legend()
plt.savefig('lasso_ridge.png',bbox_inches='tight')